In [1]:
# %load_ext autoreload
# %autoreload 2
%load_ext tuna

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import matplotlib.pyplot as plt
import numpy as np
import dill
import os
import seaborn as sns
import torch
import pandas as pd

from collections import namedtuple
from hmmlearn.hmm import MultinomialHMM
from scipy.special import logsumexp
from scipy.stats import multivariate_normal
from spn.algorithms.Inference import log_likelihood

# from rspnlib.models import RSPN, EMTrainingParams
# from rspnlib.models.hmm import HmmRSPN
# from rspnlib.models.iohmm import (
#     IoHmmRSPN,
#     ExtendedIoHmmRSPN,
#     ContinuousIoHmmRSPN,
#     OverparametrisedIoHmmRSPN,
#     OverparametrisedContinuousIoHmmRSPN,
#     ExtendedOverparametrisedContinuousIoHmmRSPN,
# )
# from rspnlib.evaluation import (
#     compute_metrics,
#     get_empty_metrics,
#     add_metrics,
#     print_aggregate_metrics,
#     evaluate_rspn_one_step_ahead_prediction,
#     evaluate_rspn_observation_regression,
# )
# from experiment_utils.datasets.vasopressors_dataset import load_vasopressors_dataset
from experiment_utils.datasets.mimic_continuous_dataset import load_mimic_dataset
# from experiment_utils.baselines.lstm import train_eval_lstm
# from experiment_utils.baselines.most_common_class import train_eval_most_common_baseline

import numpy as np

from engine_dataset import generate_engine_dataset, generate_engine_datasets

from oSLRAU_run import get_data
from spn.structure.leaves.parametric.Parametric import Gaussian, Bernoulli, Categorical,  In_Latent, Multivariate_Gaussian
from spn.structure.Base import Context
from spn.algorithms.LearningWrappers import learn_parametric
from spn.io.Graphics import plot_spn
import numpy as np
from spn.algorithms.Inference import log_likelihood
from sklearn.model_selection import train_test_split
from spn.algorithms.oSLRAU import oSLRAU, oSLRAUParams
from spn.algorithms.RSPN import RSPN
from spn.algorithms.TransformStructure import Prune ,Prune_oSLRAU

from hmmlearn import hmm
from spn.algorithms.Inference import log_likelihood

# MIMIC Experiments

In [21]:
import pickle
with open('all_mimic_vars.pickle', 'rb') as file:  
    rspn_mimic_models = pickle.load(file)  
    test_ds_list = pickle.load(file)

NUM_SEEDS = 5
SEEDS = range(42, 42 + NUM_SEEDS)

len(rspn_mimic_models), len(test_ds_list)

(5, 5)

In [52]:
%%capture
from spn.algorithms.Marginalization import marginalize
from copy import deepcopy
from spn.algorithms.MPE import mpe
from spn.structure.Base import assign_ids, rebuild_scopes_bottom_up
from spn.io.Text import str_to_spn, spn_to_str_equation
from sklearn.metrics import mean_squared_error as sk_mse

num_time_steps = 8
n_dim = 4


def next_error(model, data, num_time_steps=num_time_steps, num_dims=n_dim):
    unrolled_model = model.get_unrolled_rspn(model.get_len_sequence())
    preds_arr = np.zeros((data.shape[0], (num_time_steps)*(n_dim-1))) # assuming 1 control variable

    preds_arr[:, :3] = data[:, :3]

    p = 3 #index to insert for preds array
    for k, t in enumerate(range(8, num_time_steps*n_dim+1, 4)):
        # print(f"__________{t}___________________")

        
        #prev_inputs = list(range(3, t-4, 4))
        prev_inputs = []
        print(f"==>> prev_inputs: {prev_inputs}")


        inputs_to_include = []
        for i in range(t):
            if i not in prev_inputs:
                inputs_to_include.append(i)

        spn_marg = marginalize(unrolled_model, inputs_to_include)
        
        data_t = deepcopy(data)
        data_t[:, [t-2, t-3, t-4]] = np.nan
        #data_t = data_t[:, :t]

       
        spn_mpe = mpe(spn_marg, data_t)
        curr_spn_mpe = spn_mpe[:, [t-2, t-3, t-4]]

        print(f"==>> curr_spn_mpe[:3]: {curr_spn_mpe[:3]}")
        print(f"==>> data[:3]: {data[:3]}")
        #assert False
        #curr_data = data[:, [t-2, t-3, t-4]]
        


        preds_arr[:, [p+2, p+1, p]] = curr_spn_mpe
        p += 3

    return preds_arr



def rmse_over_vars(preds, real_data):
    rmses = []
    n_dim=4
    for i in range(n_dim-1):
        data_sl = real_data[:, i::4]
        preds_sl = preds[:, i::3]


        rmse_i = sk_mse(preds_sl.flatten(), data_sl.flatten(), squared=False)
        rmses.append(rmse_i)
    
    return rmses



def all_rmses(models, test_ds_list):
    all_rmse_vars = []
    for i, (model, test_ds) in enumerate(zip(models, test_ds_list)):
        print(f"seed:__________{SEEDS[i]}___________")
        pred_arr = next_error(model, test_ds)
        rmse_vars = rmse_over_vars(pred_arr, test_ds)
        all_rmse_vars.append(rmse_vars)
        #print(rmse_vars)
        #print(np.mean(rmse_vars))
    return all_rmse_vars
  




In [53]:
np.unique(test_ds_list[0][:,11])

array([0., 1., 2., 3.])

In [54]:
all_rmse_vars = all_rmses(rspn_mimic_models, test_ds_list)  

seed:__________42___________
==>> child: In_LatentNode_1
==>> child: In_LatentNode_2
==>> child: In_LatentNode_3
==>> child: In_LatentNode_4
==>> child: In_LatentNode_549
==>> child: In_LatentNode_550
==>> child: In_LatentNode_551
==>> child: In_LatentNode_552
==>> child: In_LatentNode_553
==>> child: In_LatentNode_554
==>> child: In_LatentNode_555
==>> child: In_LatentNode_556
==>> child: In_LatentNode_557
==>> child: In_LatentNode_558
==>> child: In_LatentNode_559
==>> child: In_LatentNode_560
==>> child: In_LatentNode_561
==>> child: In_LatentNode_562
==>> child: In_LatentNode_563
==>> child: In_LatentNode_564
==>> child: In_LatentNode_565
==>> child: In_LatentNode_566
==>> child: In_LatentNode_567
==>> child: In_LatentNode_568
==>> child: In_LatentNode_569
==>> child: In_LatentNode_570
==>> child: In_LatentNode_571
==>> child: In_LatentNode_572
==>> child: In_LatentNode_573
==>> child: In_LatentNode_574
==>> child: In_LatentNode_575
==>> child: In_LatentNode_576
==>> child: In_Late

In [57]:
# all_rmse_vars
all_rmses_arr = np.array(all_rmse_vars)
all_rmses_arr

array([[18.92325589, 16.83263931,  5.23300328],
       [13.97600297, 16.33655868,  5.06827193],
       [13.51061304, 13.78505215,  4.69203511],
       [13.46421165, 13.92399084,  4.71948141],
       [13.77872692, 15.31198438,  4.89524293]])

In [68]:
print(f"overall mean across all three variables and all 5 seeds: {np.mean(all_rmses_arr.flatten())}")
print(f"overall sd across all three variables and all 5 seeds: {np.std(all_rmses_arr.flatten())}")
print(f"mean over the 3 variables for each seed: {np.mean(all_rmses_arr, axis=1)}")
print(f"sd over the 3 variables for each seed: {np.std(all_rmses_arr, axis=1)}")
print(f"mean for each of the 3 variables over the seeds: {np.mean(all_rmses_arr, axis=0)}")
print(f"sd for each of the 3 variables over the seeds: {np.std(all_rmses_arr, axis=0)}")
print(f"full array of the rmses for each variable for each seed:\n {all_rmses_arr}")


overall mean across all three variables and all 5 seeds: 11.630071366059603
overall sd across all three variables and all 5 seeds: 4.953931764013028
mean over the 3 variables for each seed: [13.66296616 11.79361119 10.66256676 10.7025613  11.32865141]
sd over the 3 variables for each seed: [6.02167613 4.85219517 4.22328982 4.23483829 4.59196967]
mean for each of the 3 variables over the seeds: [14.73056209 15.23804507  4.92160693]
sd for each of the 3 variables over the seeds: [2.10456409 1.23230328 0.20626746]
full array of the rmses for each variable for each seed:
 [[18.92325589 16.83263931  5.23300328]
 [13.97600297 16.33655868  5.06827193]
 [13.51061304 13.78505215  4.69203511]
 [13.46421165 13.92399084  4.71948141]
 [13.77872692 15.31198438  4.89524293]]


NameError: name 'all_rmses_arr' is not defined